In [1]:
import os
import sys
sys.path.append("../")
from datatools.analyzer import *

from datatools.maneger import DataManager
import pandas as pd

/home/yamada/.local/share/virtualenvs/twitchAI-AEfI_JA6/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from wakame.tokenizer import Tokenizer
from wakame.analyzer import Analyzer
from wakame.charfilter import *
from wakame.tokenfilter import *

text = '和布ちゃんこんにちは'

# 基本的な使い方
tokenizer = Tokenizer(use_neologd=True)
tokens = tokenizer.tokenize(text)
for token in tokens:
    print(token)

和布	名詞,一般,*,*,*,*,和布,ワカメ,ワカメ
ちゃん	名詞,接尾,人名,*,*,*,ちゃん,チャン,チャン
こんにちは	感動詞,*,*,*,*,*,こんにちは,コンニチハ,コンニチワ


In [3]:
mecab_tokenize(text)

['和布', 'ちゃん', 'こんにちは']

In [5]:
text = "彼の技術は極めて優れている"

In [6]:
sentence2morpheme(text)

[['彼', 'の', '技術', 'は', '極めて', '優れ', 'て', 'いる']]

In [7]:
novel_path = "../../corpus/novel2/"

In [8]:
ncodes= [x  for x in os.listdir(novel_path) if "." not in x ]

In [9]:
def format_EOS(sentence):
    if sentence[-1] in ["。", "?", "!", "．"]:
        return sentence
    else:
         return sentence+"。"

def is_valid_utterance_segment(text:str)->bool:
    # 複数の"「"には囲まれていない
    if text.count("「") == 1 and text.count("」") == 1 and text[0]=="「" and text[-1]=="」":
        return True
    return False

In [10]:
# convs = []
    # for text in tqdm(novel):
        # if is_valid_utterance_segment(text):
        #     utt = re.search(pattern, text).group()
        #     utt = clean_text_plain(utt)
        #     utt = format_EOS( utt )
        #     convs.append(utt)
        # print(text)
    # print(convs)

In [11]:
def clean_text_plain(text):
    text_ = neologdn.normalize(text)
    # text_ = re.sub(r'\([^\)]*\)', "", text_)
    # text_ = re.sub(r'\([^\)]*\)', "", text_)
    text_ = re.sub(r'\d+', "0", text_)
    if "……" in text_:
        text_ = text_.replace("……", "…")
    return text_

In [35]:
# 1ページの小説をデータに変換
import numpy as np
import copy
pattern = '(?<=\「).*(?=\」)'
def novel2data(novel:list):
    data_list = []
    """
    1 : 会話
    0 : それ以外

    011, 101, 111 の時に登録可能とする
    
    """
    # 1, 0 の登録
    # zero_one_vector = np.zeros(len(novel))
    zero_one = ""
    novel_normalized = []

    conv_data = []

    for i, line in enumerate(novel):
        # 普通の発話なら1
        if is_valid_utterance_segment(line):
            zero_one += "1"
            utt = re.search(pattern, line).group()
            # そもそも存在しないのであれば
            if utt=="":
                # print(line)
                continue
            utt = clean_text_plain(utt)
            if utt=="":
                utt = re.search(pattern, line).group()
            utt = format_EOS( utt )
            novel_normalized.append(utt)
        else:
            zero_one += "0"
            novel_normalized.append(clean_text_plain(line))

    descriptive = []
    # 部分を検索
    for i in range(2, len(novel_normalized)):
        if zero_one[i]=="0":
            descriptive.append(novel_normalized[i])
        comp = zero_one[i-3:i]
        if "011" == comp:
            conv_data.append(descriptive[-3:] + novel_normalized[i-2:i])
            
        elif "101" == comp:
            conv_data.append(descriptive[-3:] +[novel_normalized[i-3], novel_normalized[i-1]])

        elif "111" == comp:
            conv_data.append(descriptive[-3:] + novel_normalized[i-2:i])

        else:
            pass
    
    return conv_data

In [36]:
conv_data = []

In [37]:
from tqdm import tqdm
for ncode in tqdm(ncodes):
    for number in sorted(os.listdir(novel_path+ncode) ,reverse=True):
        with open(novel_path+ncode+"/"+number, "r") as f:
            novel = f.read().splitlines()
            conv_data += novel2data(novel) 
        # break
    # break

 26%|██▌       | 131/500 [00:11<00:34, 10.61it/s]

「」


 30%|██▉       | 149/500 [00:13<00:34, 10.24it/s]

「」
「」
「」
「」


 38%|███▊      | 192/500 [00:17<00:26, 11.63it/s]

...

 73%|███████▎  | 363/500 [00:32<00:15,  8.82it/s]

「」
「」
「」


 87%|████████▋ | 434/500 [00:39<00:07,  9.17it/s]

「」
「」


100%|██████████| 500/500 [00:45<00:00, 11.01it/s]


In [39]:
len(conv_data)

1045266

In [40]:
out_path = "../../corpus/novel_formated/"
os.listdir(out_path)

[]

In [41]:
corpus_name = "novel_segments.tsv"
import csv

In [42]:
with open(out_path+corpus_name, "w") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerows(conv_data)